지역
- 양주시
- B군, 수요 데이터 없음

중심점
- 행복주택

고려한 요소들
- 공영주차장
- 쏘카존
- 지하철

스케일링 여부
- 평균으로 나눔

In [2]:
import pandas as pd
import numpy as np

import dataframe_image as dfi

In [3]:
# 데이터프레임을 그림 파일로 저장
save_option = 1 # 1 : dfi / 2 : df2img / 3 : do nothing
import df2img
def save(df, path):
    if save_option == 1:
        dfi.export(df, path)
    if save_option == 2:
        df2img.save_dataframe(fig=df2img.plot_dataframe(df), filename=path)
    if save_option == 3:
        pass

각 군집별 도시들
- cluster A : ['고양시 덕양구', '김포시', '성남시 분당구', '시흥시', '용인시 기흥구', '의정부시', '파주시']
- cluster B : ['구리시', '군포시', '성남시 수정구', '성남시 중원구', '수원시 장안구', '수원시 팔달구', '안성시', '안양시 만안구', '양주시', '오산시', '용인시 처인구', '의왕시', '이천시']
- cluster C : ['남양주시', '부천시', '화성시']
- cluster D : ['가평군', '과천시', '동두천시', '양평군', '여주시', '연천군', '포천시']
- cluster E : ['고양시 일산동구', '고양시 일산서구', '광명시', '광주시', '수원시 권선구', '수원시 영통구', '안산시 단원구', '안산시 상록구', '안양시 동안구', '용인시 수지구', '하남시']
- cluster F : ['평택시']

각 군집별 (수요 데이터가 있는) 도시들
- Cluster A : ['고양시 덕양구', '김포시', '성남시 분당구', '의정부시']
- Cluster B : ['구리시', '성남시 수정구', '성남시 중원구', '안양시 만안구']
- Cluster C : ['남양주시', '화성시']
- Cluster D : ['양평군']
- Cluster E : ['고양시 일산동구', '고양시 일산서구', '광명시', '안양시 동안구', '하남시']
- Cluster F : []

양주시는 B군에 속해있고, 수요데이터가 없는 도시이다.

# 데이터 작업

## 행복주택

In [4]:
happys = pd.read_csv("../../data/data_b_raw/2b206_0111_경기도 행복주택주거지현황.csv")
happys.rename(columns={'주소':'행복주택 주소'}, inplace=True)
happys = happys[happys['지역']=='양주시'].reset_index(drop=True)
happy_names = happys[['행복주택 주소']]
happys = happys[['경도', '위도']].rename(columns={'경도':'x', '위도':'y'})
happys = happys.values
happys.shape

(4, 2)

## 공영주차장

In [14]:
parks = pd.read_csv("../../data/data_b_raw/2b208_0202_한국교통안전공단_전국공영주차장정보_20191224.csv", encoding='CP949')
parks.rename(columns={'주차장지번주소':'주차장 주소'}, inplace=True)
parks = parks[parks['지역구분']=='경기도']
parks = parks[parks['지역구분_sub']=='오산시'].reset_index(drop=True)
park_names = parks[['주차장 주소']]
parks = parks[['경도', '위도']].rename(columns={'경도':'x','위도':'y'})
parks = parks.values
parks.shape

(31, 2)

## 쏘카존

In [10]:
# 쏘카존 갯수 데이터(김선중)
# 출처 : https://d3s0tskafalll9.cloudfront.net/media/documents/20210823_socar_zone.csv (going deeper 4-6)
zones = pd.read_csv("../../data/data_b_raw/2b201_1228_20210823_socar_zone.csv")
zones.rename(columns={'zone_name':'존 이름'}, inplace=True)
zones = zones[zones['region2']=='수원시\u3000팔달구'].reset_index(drop=True)
zone_names = zones[['존 이름']]
zones = zones[['lng','lat']]
zones.rename(columns={'lng':'x','lat':'y'}, inplace=True)
zones = zones.values
zones.shape

(20, 2)

## 지하철역

In [27]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_request")

subways = pd.read_csv("../../data/data_b_raw/2b214_0205_양주_지하철역.csv")
for i, loc in enumerate(subways['역 주소']):
    coord = geolocator.geocode(loc)[-1]
    subways.loc[i, 'x'] = coord[1]
    subways.loc[i, 'y'] = coord[0]
subway_names = subways[['역 이름']]
subways = subways[['x','y']]
subways = subways.values
subways

array([[127.03794098,  37.7755613 ],
       [127.0462671 ,  37.8212289 ],
       [127.0657869 ,  37.8454867 ]])

# 제안

In [7]:
n_happys = len(happys)
n_parks = len(parks)
n_zones = len(zones)
n_subways = len(subways)
print("n_happyhouses :",n_happys)
print("n_parks :",n_parks)
print("n_zones :",n_zones)
print("n_subways :",n_subways)

n_happyhouses : 4
n_parks : 47
n_zones : 12
n_subways : 3


In [8]:
# euclidean distance
def d(x:np.array, y:np.array) -> float:
    return np.linalg.norm(x-y)
x = np.array([0,3])
y = np.array([4,0])
d(x,y)

5.0

## score 설정

임의의 위치 $X$에 대하여, 다음과 같은 사실들을 가정한다.
- 가정1 : $X$에서 공영주차장이 가까이 위치할 수록 $X$의 근처에 쏘카존을 제안할 만하다. [subscript : P]
- 가정2 : $X$에서 쏘카존이 멀리 위치할 수록 $X$의 근처에 쏘카존을 제안할 만하다. [subscript : Z]
- 가정3 : $X$에서 지하철역이 멀리 위치할 수록 $X$의 근처에 쏘카존을 제안할 만하다. [subscript : S]
(지하철이 있으면, 지하철을 타지 쏘카를 타지는 않을 것이다.)
- 가정3 : $X$의 근처에 아파트가 많을수록 $X$의 근처에 쏘카존을 제안할 만하다. [subscript : A]
(상관분석 결과 반영)

위치 $X$에 대하여, 해당 위치 근처에 쏘카존을 제안할 만한지를 나타내는 score (score function)를 $f(X)$라고 하자.
<!-- (일반적인 목적함수 e.g. loss function/cost function/error function는 목적함수의 값을 최소화하는 것이 목적이지만, 이번 목적함수는 최대화하는 것이 목적이다.) -->

\begin{align*}
f(X)&=\alpha_P f_P(X) + \alpha_Z f_Z(X) + \alpha_S f_S(X) + \alpha_A f_A(X)\\
f_P(X)&:-(\text{가장 가까운 공영주차장까지의 거리})\\
f_Z(X)&:(\text{가장 가까운 쏘카존까지의 거리})\\
f_S(X)&:(\text{가장 가까운 지하철역까지의 거리})\\
f_A(X)&:(\text{반경 1km 이내의 아파트의 개수})
\end{align*}

단, $\alpha_P$, $\alpha_Z$, $\alpha_S$, $\alpha_A$는 각각의 additive subfunction에 대한 가중치로, 해당 feature들을 어느 정도의 중요도로 반영할지를 나타내는 값이다.
가중치가 모두 1이면, score는 다음과 같은 비교적 간단한 식이 된다.

\begin{align*}
f(X)
= &-(\text{가장 가까운 공영주차장까지의 거리}) + (\text{가장 가까운 쏘카존까지의 거리})\\
&+ (\text{가장 가까운 지하철역까지의 거리}) + (\text{반경 1km 이내의 아파트의 개수})
\end{align*}

## 계산

In [9]:
# distances : the distances of the happy house and every parking lot
# park_info : the list of (min,index) pair for every happy houses.
parks_info = []
for happy in happys:
    distances = [d(happy, park) for park in parks]
    parks_info.append(np.array([np.min(distances),np.argmin(distances)]))
parks_info = np.array(parks_info)
parks_info

array([[1.23236036e-03, 8.00000000e+00],
       [1.46534454e-02, 8.00000000e+00],
       [4.43621352e-03, 2.00000000e+00],
       [2.09499651e-03, 2.40000000e+01]])

In [11]:
parks_info_ = pd.DataFrame({'주차장 최소거리':parks_info[:,0], 'index':np.round(parks_info[:,1]).astype(int)})
parks_info_['주차장 주소'] = parks_info_['index'].replace(park_names['주차장 주소'].to_dict())
parks_info_.drop('index', axis=1, inplace=True)
parks_info_

,주차장 최소거리,주차장 주소
0,0.001232,경기도 양주시 고암동 580-7
1,0.014653,경기도 양주시 고암동 580-7
2,0.004436,경기도 양주시 광사동 688
3,0.002095,경기도 양주시 덕계동 159-3


In [12]:
# distances : the distances of the happy house and every zone
# zone_info : the list of (min,index) pair for every happy houses.
zones_info = []
for happy in happys:
    distances = [d(happy, zone) for zone in zones]
    zones_info.append(np.array([np.min(distances),np.argmin(distances)]))
zones_info = np.array(zones_info)
zones_info

array([[1.00256467e-02, 6.00000000e+00],
       [1.39793202e-02, 5.00000000e+00],
       [6.19991614e-03, 2.00000000e+00],
       [1.28791397e-02, 7.00000000e+00]])

In [13]:
zones_info_ = pd.DataFrame({'쏘카존 최소거리':zones_info[:,0], 'index':np.round(zones_info[:,1]).astype(int)})
zones_info_['존 이름'] = zones_info_['index'].replace(zone_names['존 이름'].to_dict())
zones_info_.drop('index', axis=1, inplace=True)
zones_info_

,쏘카존 최소거리,존 이름
0,0.010026,봉우재사거리 옆
1,0.013979,옥정호수도서관 앞
2,0.006200,하늘물공원 옆
3,0.012879,덕계공원


In [14]:
# distances : the distances of the happy house and every subway
# subway_info : the list of (min,index) pair for every happy houses.
subways_info = []
for happy in happys:
    distances = [d(happy, subway) for subway in subways]
    subways_info.append(np.array([np.min(distances),np.argmin(distances)]))
subways_info = np.array(subways_info)
subways_info

array([[0.01327275, 2.        ],
       [0.02480921, 2.        ],
       [0.04107845, 0.        ],
       [0.01242304, 1.        ]])

In [23]:
subway_names.to_dict()

{0: '양주', 1: '덕계', 2: '덕정'}

In [31]:
subways_info_ = pd.DataFrame({'지하철 최소거리':subways_info[:,0], 'index':np.round(subways_info[:,1]).astype(int)})
subways_info_['역 이름'] = subways_info_['index'].replace(subway_names['역 이름'].to_dict())
subways_info_.drop('index', axis=1, inplace=True)
subways_info_

,지하철 최소거리,역 이름
0,0.013273,덕정
1,0.024809,덕정
2,0.041078,양주
3,0.012423,덕계


In [34]:
scores = pd.concat([happy_names, parks_info_['주차장 최소거리'], zones_info_['쏘카존 최소거리'], subways_info_['지하철 최소거리']], axis=1)
scores['스코어'] = -scores['주차장 최소거리'] + scores['쏘카존 최소거리'] + scores['지하철 최소거리']
scores.sort_values(by='스코어', ascending=False, inplace=True)
scores = scores.rename(columns={'주차장 주소':'가까운 주차장 주소','존 이름':'가까운 존 이름','역 이름':'가까운 역 이름'})
scores

,행복주택 주소,주차장 최소거리,쏘카존 최소거리,지하철 최소거리,스코어
2,경기도 양주시 고읍남로 94,0.004436,0.006200,0.041078,0.042842
1,경기도 양주시 옥정서로 254,0.014653,0.013979,0.024809,0.024135
3,경기도 양주시 덕계로 100,0.002095,0.012879,0.012423,0.023207
0,경기도 양주시 고암길 251,0.001232,0.010026,0.013273,0.022066


In [33]:
save(scores, "results/1d204_0202_scores_양주.png")